# Sampling & Env Performance Checks
Guardrails before refactoring sampling / Env internals.


In [1]:
import os, sys, time, statistics, random
from collections import Counter
from pathlib import Path

def find_repo_root(start_dir: str) -> str:
    cur = Path(start_dir).resolve()
    for _ in range(6):
        if (cur / 'liars_poker').is_dir() or (cur / 'pyproject.toml').exists():
            return str(cur)
        if cur.parent == cur:
            break
        cur = cur.parent
    return str(Path(start_dir).resolve())

NB_DIR = Path.cwd()
REPO_ROOT = Path(find_repo_root(NB_DIR))
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
print('Repo root:', REPO_ROOT)

from liars_poker import GameSpec, Env, Rules
from liars_poker.infoset import InfoSet, CALL
from liars_poker.policies import RandomPolicy, TabularPolicy
from liars_poker.policies.tabular_dense import DenseTabularPolicy
from liars_poker.eval.match import play_match, eval_seats_split
from liars_poker.algo.br_exact import best_response_exact

rng = random.Random(123)

def freq_from_samples(samples):
    c = Counter(samples)
    total = sum(c.values())
    return {k: v / total for k, v in c.items()}

def assert_close_dist(got, expected, tol=0.08):
    for action, exp in expected.items():
        g = got.get(action, 0.0)
        assert abs(g - exp) <= tol, f"{action}: {g:.3f} vs {exp:.3f}"

def bench(fn, runs=5, warmup=1):
    for _ in range(warmup):
        fn()
    times = []
    for _ in range(runs):
        t0 = time.perf_counter()
        fn()
        times.append(time.perf_counter() - t0)
    return times

def summarize(times):
    if not times:
        return None
    return {
        'mean': statistics.mean(times),
        'stdev': statistics.stdev(times) if len(times) > 1 else 0.0,
        'median': statistics.median(times),
        'min': min(times),
        'max': max(times),
    }

def report(label, times, units='s', per=None):
    stats = summarize(times)
    if stats is None:
        return
    mean = stats['mean']
    extra = ''
    if per is not None and per > 0:
        extra = f", per={mean / per:.6f} {units}"
    print(f"{label}: mean={mean:.6f}{units} std={stats['stdev']:.6f}{units} median={stats['median']:.6f}{units}{extra}")


Repo root: C:\Users\adidh\Documents\liars_poker


## Correctness: Sampling determinism
Ensure sampling respects RNG and legal actions.


In [2]:
spec = GameSpec(ranks=3, suits=1, hand_size=1, claim_kinds=('RankHigh',))
rules = Rules(spec)
pol = RandomPolicy(); pol.bind_rules(rules)
iset = InfoSet(pid=0, hand=(1,), history=())
legal = rules.legal_actions_for(iset)
print('Legal actions:', legal)

rng1 = random.Random(1)
samples1 = [pol.sample(iset, rng1) for _ in range(8)]
rng2 = random.Random(1)
samples2 = [pol.sample(iset, rng2) for _ in range(8)]
print('Samples1 == Samples2:', samples1 == samples2, samples1)
assert all(a in legal for a in samples1)
assert samples1 == samples2
print('Deterministic sampling OK.')


Legal actions: (0, 1, 2)
Samples1 == Samples2: True [0, 2, 2, 0, 1, 1, 1, 2]
Deterministic sampling OK.


## Correctness: Sampling distribution vs action_probs
Empirically check that sampling matches intended probabilities.


In [3]:
spec_dist = GameSpec(ranks=3, suits=1, hand_size=1, claim_kinds=('RankHigh',))
rules_dist = Rules(spec_dist)
iset = InfoSet(pid=0, hand=(1,), history=())
legal = rules_dist.legal_actions_for(iset)

tab = TabularPolicy(); tab.bind_rules(rules_dist)
tab.set(iset, {legal[0]: 0.7, legal[1]: 0.2, legal[2]: 0.1})
expected = tab.action_probs(iset)

rng = random.Random(5)
samples = [tab.sample(iset, rng) for _ in range(5000)]
freq = freq_from_samples(samples)
print('Tabular expected:', expected)
print('Tabular sampled:', {k: round(v,3) for k,v in freq.items()})
assert_close_dist(freq, expected, tol=0.07)

dense = DenseTabularPolicy(spec_dist)
dense.S[0, dense.hand_to_idx[iset.hand], :] = 0.0
dense.S[0, dense.hand_to_idx[iset.hand], 1] = 0.7
dense.S[0, dense.hand_to_idx[iset.hand], 2] = 0.2
dense.S[0, dense.hand_to_idx[iset.hand], 3] = 0.1
dense.recompute_likelihoods()
expected_dense = dense.action_probs(iset)
rng = random.Random(5)
samples = [dense.sample(iset, rng) for _ in range(5000)]
freq = freq_from_samples(samples)
print('Dense expected:', expected_dense)
print('Dense sampled:', {k: round(v,3) for k,v in freq.items()})
assert_close_dist(freq, expected_dense, tol=0.07)
print('Distribution checks OK.')


Tabular expected: {0: 0.7, 1: 0.2, 2: 0.1}
Tabular sampled: {0: 0.703, 1: 0.194, 2: 0.103}
Dense expected: {0: 0.699999988079071, 1: 0.20000000298023224, 2: 0.10000000149011612}
Dense sampled: {0: 0.703, 1: 0.194, 2: 0.103}
Distribution checks OK.


## Correctness: Dense vs Tabular defaults
Uniform defaults should match for the same infoset/history.


In [4]:
spec_def = GameSpec(ranks=3, suits=2, hand_size=1, claim_kinds=('RankHigh','Pair'), suit_symmetry=True)
rules_def = Rules(spec_def)
tab = TabularPolicy(); tab.bind_rules(rules_def)
dense = DenseTabularPolicy(spec_def)

iset_root = InfoSet(pid=0, hand=(1,), history=())
iset_after = InfoSet(pid=1, hand=(1,), history=(0,))

tab_root = tab.action_probs(iset_root)
dense_root = dense.action_probs(iset_root)
tab_after = tab.action_probs(iset_after)
dense_after = dense.action_probs(iset_after)

assert_close_dist(tab_root, dense_root, tol=1e-6)
assert_close_dist(tab_after, dense_after, tol=1e-6)
print('Defaults match for root and post-claim states.')


Defaults match for root and post-claim states.


## Correctness: Eval helpers invariants
Sanity-check seat split results and play_match.


In [5]:
spec_sm = GameSpec(ranks=2, suits=1, hand_size=1, claim_kinds=('RankHigh',))
rules_sm = Rules(spec_sm)
p_rand = RandomPolicy(); p_rand.bind_rules(rules_sm)
p_br, _ = best_response_exact(spec_sm, p_rand)
p_br.bind_rules(rules_sm)

wins = play_match(Env(spec_sm), p_rand, p_br, episodes=40, seed=7)
print('Sampled wins (rand vs BR):', wins)

seats = eval_seats_split(spec_sm, p_rand, p_br, episodes=200, seed=9)
print('Seat-split win rates:', seats)
assert abs((seats['A_seat1'] + seats['B_seat2']) - 1.0) < 1e-9
assert abs((seats['A_seat2'] + seats['B_seat1']) - 1.0) < 1e-9
print('Seat-split invariants OK.')


Sampled wins (rand vs BR): {'P1': 16, 'P2': 24}
Seat-split win rates: {'A_seat1': 0.49, 'A_seat2': 0.0, 'B_seat1': 1.0, 'B_seat2': 0.51}
Seat-split invariants OK.


## Performance: policy.sample microbench
Measure per-sample runtime for common policy types.


In [6]:
spec_bench = GameSpec(ranks=3, suits=2, hand_size=1, claim_kinds=('RankHigh','Pair'), suit_symmetry=True)
rules_bench = Rules(spec_bench)
iset = InfoSet(pid=0, hand=(1,), history=())

rand_pol = RandomPolicy(); rand_pol.bind_rules(rules_bench)
tab_pol = TabularPolicy(); tab_pol.bind_rules(rules_bench)
tab_pol.set(iset, {0: 0.6, 1: 0.2, 2: 0.2})
dense_pol = DenseTabularPolicy(spec_bench)
dense_pol.S[0, dense_pol.hand_to_idx[iset.hand], :] = 0.0
dense_pol.S[0, dense_pol.hand_to_idx[iset.hand], 1] = 0.6
dense_pol.S[0, dense_pol.hand_to_idx[iset.hand], 2] = 0.2
dense_pol.S[0, dense_pol.hand_to_idx[iset.hand], 3] = 0.2
dense_pol.recompute_likelihoods()

N = 50000

def bench_sample(policy):
    r = random.Random(1)
    for _ in range(N):
        policy.sample(iset, r)

for label, policy in [('Random', rand_pol), ('Tabular', tab_pol), ('Dense', dense_pol)]:
    times = bench(lambda p=policy: bench_sample(p), runs=5, warmup=1)
    report(f'{label} sample', times, units='s', per=N)


Random sample: mean=0.446815s std=0.009302s median=0.449242s, per=0.000009 s
Tabular sample: mean=0.715074s std=0.053046s median=0.711724s, per=0.000014 s
Dense sample: mean=0.367926s std=0.001001s median=0.367808s, per=0.000007 s


## Performance: play_match microbench
Measure end-to-end match runtime for common policy pairings.


In [7]:
spec_match = GameSpec(ranks=3, suits=2, hand_size=1, claim_kinds=('RankHigh','Pair'), suit_symmetry=True)
rules_match = Rules(spec_match)
rand_a = RandomPolicy(); rand_a.bind_rules(rules_match)
rand_b = RandomPolicy(); rand_b.bind_rules(rules_match)
dense_a = DenseTabularPolicy(spec_match)
dense_b = DenseTabularPolicy(spec_match)

EP = 200

def bench_play(p1, p2):
    env = Env(spec_match)
    play_match(env, p1, p2, episodes=EP, seed=77)

pairs = [
    ('Rand vs Rand', rand_a, rand_b),
    ('Rand vs Dense', rand_a, dense_a),
    ('Dense vs Dense', dense_a, dense_b),
]

for label, p1, p2 in pairs:
    times = bench(lambda a=p1, b=p2: bench_play(a, b), runs=5, warmup=1)
    report(label, times, units='s', per=EP)


Rand vs Rand: mean=0.009850s std=0.000012s median=0.009856s, per=0.000049 s
Rand vs Dense: mean=0.011031s std=0.000306s median=0.010897s, per=0.000055 s
Dense vs Dense: mean=0.012268s std=0.000059s median=0.012263s, per=0.000061 s


In [8]:
from liars_poker.serialization import load_policy

pol_a, spec_match = load_policy('C:/Users/adidh/Documents/liars_poker/artifacts/benchmark_runs/r5_s2_h1_h_ss___20260102-045537/policy')


In [9]:
rules_match = Rules(spec_match)
pol_a.bind_rules(rules_match)
rand_b = RandomPolicy(); rand_b.bind_rules(rules_match)


EP = 1000

def bench_play(p1, p2):
    env = Env(spec_match)
    play_match(env, p1, p2, episodes=EP, seed=77)

pairs = [
    ('Rand vs pol_a', pol_a, rand_b),
]

for label, p1, p2 in pairs:
    times = bench(lambda a=p1, b=p2: bench_play(a, b), runs=5, warmup=1)
    report(label, times, units='s', per=EP)


Rand vs pol_a: mean=0.056633s std=0.001771s median=0.056024s, per=0.000057 s


In [10]:
# Fast-path sanity check (slow vs fast rollout win rates)
from liars_poker.eval.match import play_match

spec = GameSpec(ranks=3, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True)
env = Env(spec, seed=123)
p1 = RandomPolicy(); p1.bind_rules(env.rules)
p2 = RandomPolicy(); p2.bind_rules(env.rules)

wins = play_match(env, p1, p2, episodes=2000, seed=999)
rate_p1 = wins["P1"] / max(1, sum(wins.values()))
rate_p2 = wins["P2"] / max(1, sum(wins.values()))
print(wins, f"P1={rate_p1:.3f} P2={rate_p2:.3f}")
assert abs(rate_p1 - rate_p2) < 0.08


{'P1': 1074, 'P2': 926} P1=0.537 P2=0.463


In [12]:
# Fast vs slow rollout timing (rough comparison)
import time
import random

def slow_play_match(env, p1, p2, episodes=1000, seed=0):
    rng = random.Random(seed)
    wins = {"P1": 0, "P2": 0}
    p1.bind_rules(env.rules)
    p2.bind_rules(env.rules)
    for _ in range(episodes):
        obs = env.reset(seed=rng.randint(0, 2_147_483_647))
        p1.begin_episode(rng)
        p2.begin_episode(rng)
        while True:
            if obs["terminal"]:
                winner = obs["winner"]
                if winner in wins:
                    wins[winner] += 1
                break
            player = env.current_player()
            policy = p1 if player == "P1" else p2
            infoset = env.infoset_key(player)
            action = policy.sample(infoset, rng)
            obs = env.step(action)
    return wins

spec = GameSpec(ranks=7, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True)
episodes = 3000
repeats = 3

def bench(fn):
    times = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        fn()
        times.append(time.perf_counter() - t0)
    return sum(times) / len(times)

fast_env = Env(spec)
slow_env = Env(spec)

def run_fast():
    p1 = RandomPolicy()
    p2 = RandomPolicy()
    play_match(fast_env, p1, p2, episodes=episodes, seed=123)

def run_slow():
    p1 = RandomPolicy()
    p2 = RandomPolicy()
    slow_play_match(slow_env, p1, p2, episodes=episodes, seed=123)

fast_time = bench(run_fast)
slow_time = bench(run_slow)
speedup = slow_time / fast_time if fast_time > 0 else float('inf')
print(f"fast={fast_time:.3f}s slow={slow_time:.3f}s speedup={speedup:.2f}x")


fast=0.169s slow=0.326s speedup=1.92x


In [13]:
# More representative benchmark: DenseTabularPolicy vs DenseTabularPolicy
import time
import random

def slow_play_match(env, p1, p2, episodes=3000, seed=0):
    rng = random.Random(seed)
    wins = {"P1": 0, "P2": 0}
    p1.bind_rules(env.rules)
    p2.bind_rules(env.rules)
    for _ in range(episodes):
        obs = env.reset(seed=rng.randint(0, 2_147_483_647))
        p1.begin_episode(rng)
        p2.begin_episode(rng)
        while True:
            if obs["terminal"]:
                wins[obs["winner"]] += 1
                break
            player = env.current_player()
            policy = p1 if player == "P1" else p2
            infoset = env.infoset_key(player)
            action = policy.sample(infoset, rng)
            obs = env.step(action)
    return wins

spec = GameSpec(ranks=6, suits=3, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True)
fast_env = Env(spec)
slow_env = Env(spec)

p_fast = DenseTabularPolicy(spec)
p_slow = DenseTabularPolicy(spec)

episodes = 4000
repeats = 3

def bench(fn):
    times = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        fn()
        times.append(time.perf_counter() - t0)
    return sum(times) / len(times)

def run_fast():
    play_match(fast_env, p_fast, p_fast, episodes=episodes, seed=123)

def run_slow():
    slow_play_match(slow_env, p_slow, p_slow, episodes=episodes, seed=123)

fast_time = bench(run_fast)
slow_time = bench(run_slow)
print(f"fast={fast_time:.3f}s slow={slow_time:.3f}s speedup={slow_time/fast_time:.2f}x")


fast=0.325s slow=0.515s speedup=1.59x


In [15]:
# Benchmark: Fast vs slow rollout timing using *TabularPolicy* with ~1000 manual probability edits
import time
import random
import numpy as np

from liars_poker.core import GameSpec, possible_starting_hands
from liars_poker.env import Env, Rules
from liars_poker.infoset import InfoSet, CALL, NO_CLAIM
from liars_poker.policies.random import RandomPolicy
from liars_poker.policies.tabular import TabularPolicy



# ---- Slow reference match loop (unchanged) ----
def slow_play_match(env, p1, p2, episodes=1000, seed=0):
    rng = random.Random(seed)
    wins = {"P1": 0, "P2": 0}
    p1.bind_rules(env.rules)
    p2.bind_rules(env.rules)
    for _ in range(episodes):
        obs = env.reset(seed=rng.randint(0, 2_147_483_647))
        p1.begin_episode(rng)
        p2.begin_episode(rng)
        while True:
            if obs["terminal"]:
                winner = obs["winner"]
                if winner in wins:
                    wins[winner] += 1
                break
            player = env.current_player()
            policy = p1 if player == "P1" else p2
            infoset = env.infoset_key(player)
            action = policy.sample(infoset, rng)
            obs = env.step(action)
    return wins


# ---- Build a TabularPolicy and apply ~n_edits manual edits to .probs ----
def random_history_nonterminal(rules: Rules, rng: random.Random, max_len: int) -> tuple[int, ...]:
    """
    Generate a random *nonterminal* public history (no CALL), respecting strictly-increasing claim indices.
    """
    history: list[int] = []
    last_claim = NO_CLAIM
    for _ in range(max_len):
        last_idx = None if last_claim == NO_CLAIM else last_claim
        legal = rules.legal_actions_from_last(last_idx)
        # Exclude CALL so history stays nonterminal
        legal_noncall = [a for a in legal if a != CALL]
        if not legal_noncall:
            break
        a = rng.choice(legal_noncall)
        history.append(a)
        last_claim = a
    return tuple(history)


def make_edited_tabular_policy(
    spec: GameSpec,
    *,
    n_edits: int = 1000,
    seed: int = 0,
    max_history_len: int = 12,
) -> TabularPolicy:
    """
    Start from a blank TabularPolicy (defaults to uniform when missing),
    then write ~n_edits infoset distributions into .probs.
    """
    rng = random.Random(seed)
    rules = Rules(spec)

    pol = TabularPolicy()
    pol.bind_rules(rules)

    hands = list(possible_starting_hands(spec))

    # Precompute all possible opening claim actions (no CALL on first move)
    k = len(rules.claims)

    for _ in range(n_edits):
        # Random nonterminal history; derive pid from parity of history length
        hist = random_history_nonterminal(rules, rng, max_len=max_history_len)
        pid = len(hist) % 2

        # Random hand
        hand = rng.choice(hands)
        iset = InfoSet(pid=pid, hand=hand, history=hist)

        legal = rules.legal_actions_for(iset)
        if not legal:
            continue

        # Create a non-uniform distribution over legal actions
        # (Dirichlet gives random probabilities, then we sparsify a bit)
        w = np.asarray([rng.random() + 0.01 for _ in legal], dtype=float)

        # Sparsify ~40% of actions to 0 to mimic realistic tabular sparsity
        if len(legal) >= 3:
            mask = np.asarray([rng.random() > 0.4 for _ in legal], dtype=bool)
            if not mask.any():
                mask[rng.randrange(len(legal))] = True
            w = w * mask

        if w.sum() <= 0:
            # fallback: deterministic on a random legal
            dist = {rng.choice(legal): 1.0}
        else:
            w = w / w.sum()
            dist = {a: float(p) for a, p in zip(legal, w) if p > 0}

        pol.set(iset, dist)

    return pol


# ---- Benchmark harness ----
spec = GameSpec(ranks=7, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True)
episodes = 3000
repeats = 3
n_edits = 1000

fast_env = Env(spec)
slow_env = Env(spec)

def bench(fn):
    times = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        fn()
        times.append(time.perf_counter() - t0)
    return sum(times) / len(times)

def run_fast():
    # Build two tabular policies with manual edits
    p1 = make_edited_tabular_policy(spec, n_edits=n_edits, seed=1)
    p2 = make_edited_tabular_policy(spec, n_edits=n_edits, seed=2)
    play_match(fast_env, p1, p2, episodes=episodes, seed=123)

def run_slow():
    p1 = make_edited_tabular_policy(spec, n_edits=n_edits, seed=1)
    p2 = make_edited_tabular_policy(spec, n_edits=n_edits, seed=2)
    slow_play_match(slow_env, p1, p2, episodes=episodes, seed=123)

fast_time = bench(run_fast)
slow_time = bench(run_slow)
speedup = slow_time / fast_time if fast_time > 0 else float("inf")
print(f"Tabular edited (~{n_edits} sets) :: fast={fast_time:.3f}s slow={slow_time:.3f}s speedup={speedup:.2f}x")


Tabular edited (~1000 sets) :: fast=0.272s slow=0.426s speedup=1.57x
